## Model Training



In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
import warnings
warnings.filterwarnings(action='ignore')
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dot, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from wordcloud import WordCloud
from collections import defaultdict
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [34]:
# load dataset with only relevant columns
df=pd.read_csv('./AniMate_Model/rating_data.csv', usecols=["user_id","anime_id","rating"])
print("Shape of the Dataset:",df.shape)
df.head()

Shape of the Dataset: (24325191, 3)


,user_id,anime_id,rating
0,1,21,9
1,1,48,7
2,1,320,5
3,1,49,8
4,1,304,8


In [35]:
# printing duplicate rows for inspection
duplicated_rows = df[df.duplicated()]
print("Duplicated Rows:")
print(duplicated_rows)

Duplicated Rows:
Empty DataFrame
Columns: [user_id, anime_id, rating]
Index: []


# Data Preprocessing

In [36]:
scaler = MinMaxScaler(feature_range=(0, 1))
# Converting all ratings in a range of 0-1 which makes training easier and more stable
df['scaled_score'] = scaler.fit_transform(df[['rating']])

In [37]:
# converting users and animes with random ids to ordered ids starting from 0
user_encoder = LabelEncoder()
df["user_encoded"] = user_encoder.fit_transform(df["user_id"])
num_users = len(user_encoder.classes_)
anime_encoder = LabelEncoder()
df["anime_encoded"] = anime_encoder.fit_transform(df["anime_id"])
num_animes = len(anime_encoder.classes_)
print("Number of unique users: {}, Number of unique anime: {}".format(num_users, num_animes))
print("Minimum rating: {}, Maximum rating: {}".format(min(df['rating']), max(df['rating'])))

Number of unique users: 270033, Number of unique anime: 16500
Minimum rating: 1, Maximum rating: 10


# Model training

In [38]:
# shuffling the dataset randomly
df = shuffle(df, random_state=100)
X = df[['user_encoded', 'anime_encoded']].values
y = df["scaled_score"].values

In [39]:
test_set_size = 10000
# splitting dataset into train and test set with a fixed size for the test dataset size
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_set_size, random_state=73)

In [40]:
# X array contains two inputs: user_id and anime_id but label encoded
X_train_array = [X_train[:, 0], X_train[:, 1]]
X_test_array = [X_test[:, 0], X_test[:, 1]]

In [41]:
def NN_Recommender(num_users, num_animes, embedding_size=128):
    # inputting the label encoded user id
    user = Input(name='user_encoded', shape=[1])
    # convert the user id into embeddings
    user_embedding = Embedding(name='user_embedding', input_dim=num_users, output_dim=embedding_size)(user)

    # inputting the label encoded anime id
    anime = Input(name='anime_encoded', shape=[1])
    # convert the anime id into embeddings
    anime_embedding = Embedding(name='anime_embedding', input_dim=num_animes, output_dim=embedding_size)(anime)

    # finding the cosine similarily between user's and anime's latent feature space
    dot_product = Dot(name='dot_product', normalize=True, axes=2)([user_embedding, anime_embedding])
    # flattening the similarity calculation so that they can be sent to the dense layers
    flattened = Flatten()(dot_product)

    # layer for learning complex patterns in the latent space similarity scores
    dense = Dense(64, activation='relu')(flattened)
    # output layer for predicting the scaled rating (between 0 and 1)
    output = Dense(1, activation='sigmoid')(dense)

    # providing the inputs and outputs to the model
    model = Model(inputs=[user, anime], outputs=output)
    # compiling model with mse error for regression and adam optimizer with a standard learning rate
    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001), metrics=["mae", "mse"])

    return model

# creating the model
model = NN_Recommender(num_users, num_animes)

In [44]:
# code to save the model weights
from tensorflow.keras.callbacks import ModelCheckpoint
batch_size = 10000

checkpoint_filepath = './AniMate_Model/model.weights.h5'

model_checkpoints = ModelCheckpoint(filepath=checkpoint_filepath,
                                    save_weights_only=True,
                                    monitor='val_loss',
                                    mode='min',
                                    save_best_only=True)

my_callbacks = [
    model_checkpoints
]

In [ ]:
# training the model
history = model.fit(
    x=X_train_array,
    y=y_train,
    batch_size=batch_size,
    epochs=20,
    verbose=1,
    validation_data=(X_test_array, y_test),
    callbacks=my_callbacks,
)

In [43]:
# loading weights from a trained checkpoint file
model.load_weights(checkpoint_filepath)

# Recommendation:

In [45]:
def extract_weights(name, model):
    # gets the layer
    weight_layer = model.get_layer(name)
    # gets that layer's weight matrix
    weights = weight_layer.get_weights()[0]
    # computes L2 norm and normalizes the weights using it
    weights = weights / np.linalg.norm(weights, axis=1).reshape((-1, 1))
    return weights

# extracts weights for anime and user embedding layers
anime_weights = extract_weights('anime_embedding', model)
user_weights = extract_weights('user_embedding', model)

In [46]:
df_anime=pd.read_csv('./AniMate_Model/anime_data.csv')

In [47]:
# extracts only those anime with more than 50 members
popularity_threshold = 50
df_anime= df_anime.query('Members >= @popularity_threshold')
print(df_anime.shape)
df_anime.head(3)

(22879, 24)


,anime_id,Name,English name,Other name,Score,Genres,Synopsis,Type,Episodes,Aired,...,Studios,Source,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL
0,1,Cowboy Bebop,Cowboy Bebop,カウボーイビバップ,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity has expanded across the galaxy, filling the surface of other planets with settlements like those on Earth. These new societies are plagued by murder, drug use, and theft, and intergalactic outlaws are hunted by a growing number of tough bounty hunters.\n\nSpike Spiegel and Jet Black pursue criminals throughout space to make a humble living. Beneath his goofy and aloof demeanor, Spike is haunted by the weight of his violent past. Meanwhile, Jet manages his own troubled memories while taking care of Spike and the Bebop, their ship. The duo is joined by the beautiful con artist Faye Valentine, odd child Edward Wong Hau Pepelu Tivrusky IV, and Ein, a bioengineered Welsh Corgi.\n\nWhile developing bonds and working to catch a colorful cast of criminals, the Bebop crew's lives are disrupted by a menace from Spike's past. As a rival's maniacal plot continues to unravel, Spike must choose between life with his newfound family or revenge for his old wounds.",TV,26,"Apr 3, 1998 to Apr 24, 1999",...,Sunrise,Original,24 min per ep,R - 17+ (violence & profanity),41,43,78525,914193,1771505,https://cdn.myanimelist.net/images/anime/4/19644.jpg
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,カウボーイビバップ 天国の扉,8.38,"Action, Sci-Fi","Another day, another bounty—such is the life of the often unlucky crew of the Bebop. However, this routine is interrupted when Faye, who is chasing a fairly worthless target on Mars, witnesses an oil tanker suddenly explode, causing mass hysteria. As casualties mount due to a strange disease spreading through the smoke from the blast, a whopping three hundred million woolong price is placed on the head of the supposed perpetrator.\n\nWith lives at stake and a solution to their money problems in sight, the Bebop crew springs into action. Spike, Jet, Faye, and Edward, followed closely by Ein, split up to pursue different leads across Alba City. Through their individual investigations, they discover a cover-up scheme involving a pharmaceutical company, revealing a plot that reaches much further than the ragtag team of bounty hunters could have realized.",Movie,1,1-Sep-01,...,Bones,Original,1 hr 55 min,R - 17+ (violence & profanity),189,602,1448,206248,360978,https://cdn.myanimelist.net/images/anime/1439/93480.jpg
2,6,Trigun,Trigun,トライガン,8.22,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,000,000 bounty on his head. The reason: he's a merciless villain who lays waste to all those that oppose him and flattens entire cities for fun, garnering him the title ""The Humanoid Typhoon."" He leaves a trail of death and destruction wherever he goes, and anyone can count themselves dead if they so much as make eye contact—or so the rumors say. In actuality, Vash is a huge softie who claims to have never taken a life and avoids violence at all costs.\n\nWith his crazy doughnut obsession and buffoonish attitude in tow, Vash traverses the wasteland of the planet Gunsmoke, all the while followed by two insurance agents, Meryl Stryfe and Milly Thompson, who attempt to minimize his impact on the public. But soon, their misadventures evolve into life-or-death situations as a group of legendary assassins are summoned to bring about suffering to the trio. Vash's agonizing past will be unraveled and his morality and principles pushed to the breaking point.",TV,26,"Apr 1, 1998 to Sep 30, 1998",...,Madhouse,Manga,24 min per ep,PG-13 - Teens 13 or older,328,246,15035,356739,727252,https://cdn.myanimelist.net/images/anime/7/20310.jpg


# 1: Item Based Recommendation

In [48]:
def find_similar_animes(name, n=10, return_dist=False, neg=False):
    try:
        # find the anime's data from it's name
        anime_row = df_anime[df_anime['Name'] == name].iloc[0]
        index = anime_row['anime_id']
        # convert into encoded value using the Label encoder from before
        encoded_index = anime_encoder.transform([index])[0]
        # get the anime weights
        weights = anime_weights
        # calculate dot product with weights of every other anime
        dists = np.dot(weights, weights[encoded_index])
        # sort by similarity
        sorted_dists = np.argsort(dists)
        # highest similarity would be with itself so would need 10 more anime for recommendations hence the +1
        n = n + 1
        # send most similar or least similar??
        if neg:
            closest = sorted_dists[:n]
        else:
            closest = sorted_dists[-n:]
        print('Animes closest to {}'.format(name))
        # condition for if we ever need to return the distances as well
        if return_dist:
            return dists, closest

        SimilarityArr = []

        # for every similar anime we extract it's data, convert it's similarity into percentage, and then add to the final result dataframe
        for close in closest:
            decoded_id = anime_encoder.inverse_transform([close])[0]
            anime_frame = df_anime[df_anime['anime_id'] == decoded_id]

            anime_name = anime_frame['Name'].values[0]
            english_name = anime_frame['English name'].values[0]
            name = english_name if english_name != "UNKNOWN" else anime_name
            genre = anime_frame['Genres'].values[0]
            Synopsis = anime_frame['Synopsis'].values[0]
            similarity = dists[close]
            similarity = "{:.2f}%".format(similarity * 100)
            SimilarityArr.append({"Name": name, "Similarity": similarity, "Genres": genre, "Synopsis":Synopsis})
        Frame = pd.DataFrame(SimilarityArr).sort_values(by="Similarity", ascending=False)
        return Frame[Frame.Name != name]
    except:
        print('{} not found in Anime list'.format(name))

pd.set_option('display.max_colwidth', None)

# 2: User Based Recommendation


In [53]:
# exact same process for the user as well
def find_similar_users(item_input, n=10, return_dist=False, neg=False):
    try:
        index = item_input
        encoded_index = user_encoder.transform([index])[0]
        weights = user_weights
        dists = np.dot(weights, weights[encoded_index])
        sorted_dists = np.argsort(dists)
        n = n + 1

        if neg:
            closest = sorted_dists[:n]
        else:
            closest = sorted_dists[-n:]

        SimilarityArr = []

        for close in closest:
            similarity = dists[close]
            if isinstance(item_input, int):
                decoded_id = user_encoder.inverse_transform([close])[0]
                SimilarityArr.append({"similar_users": decoded_id, "similarity": similarity})
        Frame = pd.DataFrame(SimilarityArr).sort_values(by="similarity", ascending=False)
        return Frame
    except:
        print('\033[1m{}\033[0m, Not Found in User list'.format(item_input))

In [50]:
ratings_per_user = df.groupby('user_id').size()
random_user = int(ratings_per_user[ratings_per_user < 500].sample(1, random_state=None).index[0])
similar_users = find_similar_users(random_user, n=10, neg=False)
similar_users = similar_users[similar_users.similarity > 0.4]
similar_users = similar_users[similar_users.similar_users != random_user]
similar_users

,similar_users,similarity
9,424083,0.554121
8,790209,0.490223
7,503935,0.489831
6,374539,0.487562
5,1140719,0.479501
4,457961,0.478378
3,472708,0.470432
2,502117,0.468337
1,497080,0.467882
0,85472,0.466369


In [56]:
def get_user_preferences(user_id, plot=False, verbose=0):
    # get all rows where user has given a rating
    animes_watched_by_user = df[df['user_id'] == user_id]

    if animes_watched_by_user.empty:
        print("User #{} has not watched any animes.".format(user_id))
        return pd.DataFrame()

    # keep only the top 25% of the highest rated anime by the user
    user_rating_percentile = np.percentile(animes_watched_by_user.rating, 75)
    animes_watched_by_user = animes_watched_by_user[animes_watched_by_user.rating >= user_rating_percentile]
    # sort those anime by their ratings
    top_animes_user = (
        animes_watched_by_user.sort_values(by="rating", ascending=False)
        .anime_id.values
    )

    # extract those anime's name and genres
    anime_df_rows = df_anime[df_anime["anime_id"].isin(top_animes_user)]
    anime_df_rows = anime_df_rows[["Name", "Genres"]]

    # print details if needed
    if verbose != 0:
        print("User \033[1m{}\033[0m has watched {} anime(s) with an average rating of {:.1f}/10\n".format(
            user_id, len(animes_watched_by_user), animes_watched_by_user['rating'].mean()
        ))
        print('\033[1m----- Preferred genres----- \033[0m\n')

    # plot if needed
    if plot:
        genres_list = []
        for genres in anime_df_rows['Genres']:
            if isinstance(genres, str):
                for genre in genres.split(','):
                    genres_list.append(genre.strip())

    return anime_df_rows

In [58]:
get_user_preferences(1, plot=True, verbose=1)

User 1 has watched 134 anime(s) with an average rating of 8.5/10

----- Preferred genres----- 



,Name,Genres
0,Cowboy Bebop,"Action, Award Winning, Sci-Fi"
1,Cowboy Bebop: Tengoku no Tobira,"Action, Sci-Fi"
2,Trigun,"Action, Adventure, Sci-Fi"
3,Witch Hunter Robin,"Action, Drama, Mystery, Supernatural"
6,Hachimitsu to Clover,"Comedy, Drama, Romance"
...,...,...
6714,Ookami Kodomo no Ame to Yuki,"Award Winning, Fantasy, Slice of Life"
7428,Shingeki no Kyojin,"Action, Award Winning, Drama, Suspense"
9352,Shingeki no Kyojin Season 2,"Action, Drama, Suspense"
10408,One Punch Man,"Action, Comedy"


In [68]:
def get_recommended_animes(similar_users, user_pref, n=10):
    recommended_animes = []
    anime_list = []

    for user_id in similar_users.similar_users.values:
        # get each similar user's top rated animes
        pref_list = get_user_preferences(int(user_id))
        # exclude those which the current user has already liked and add the remaining to anime list
        if not pref_list.empty:
            pref_list = pref_list[~pref_list["Name"].isin(user_pref["Name"].values)]
            anime_list.append(pref_list.Name.values)

    if len(anime_list) == 0:
        print("No anime recommendation.")
        return pd.DataFrame()

    # count which anime have been recommended multiple time and then rank them in descending order
    anime_list = pd.DataFrame(anime_list)
    sorted_list = pd.DataFrame(pd.Series(anime_list.values.ravel()).value_counts()).head(n)
    # count how many users from the entire dataset have rated each of these anime (global popularity)
    anime_count = df['anime_id'].value_counts()

    # store anime's data in the final result based on how many people have rated them globally descending order
    for i, anime_name in enumerate(sorted_list.index):
        if isinstance(anime_name, str):
            try:
                anime_id = df_anime[df_anime.Name == anime_name].anime_id.values[0]
                english_name = df_anime[df_anime['Name'] == anime_name]['English name'].values[0]
                name = english_name if english_name != "UNKNOWN" else anime_name
                genre = df_anime[df_anime.Name == anime_name].Genres.values[0]
                Synopsis = df_anime[df_anime.Name == anime_name].Synopsis.values[0]
                n_user_pref = anime_count.get(anime_id, 0)
                recommended_animes.append({
                    "n": n_user_pref,
                    "anime_name": anime_name,
                    "Genres": genre,
                    "Synopsis": Synopsis
                })
            except:
                pass
    return pd.DataFrame(recommended_animes)

In [67]:
random_user = 1
similar_users = find_similar_users(random_user)
user_pref = get_user_preferences(random_user)
recommendations = get_recommended_animes(similar_users, user_pref, n=10)
print('\033[1m----- Recommended animes ----- \033[0m\n')
recommendations

----- Recommended animes ----- 



,n,anime_name,Genres,Synopsis
0,76710,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy","After a horrific alchemy experiment goes wrong in the Elric household, brothers Edward and Alphonse are left in a catastrophic new reality. Ignoring the alchemical principle banning human transmutation, the boys attempted to bring their recently deceased mother back to life. Instead, they suffered brutal personal loss: Alphonse's body disintegrated while Edward lost a leg and then sacrificed an arm to keep Alphonse's soul in the physical realm by binding it to a hulking suit of armor.\n\nThe brothers are rescued by their neighbor Pinako Rockbell and her granddaughter Winry. Known as a bio-mechanical engineering prodigy, Winry creates prosthetic limbs for Edward by utilizing ""automail,"" a tough, versatile metal used in robots and combat armor. After years of training, the Elric brothers set off on a quest to restore their bodies by locating the Philosopher's Stone—a powerful gem that allows an alchemist to defy the traditional laws of Equivalent Exchange.\n\nAs Edward becomes an infamous alchemist and gains the nickname ""Fullmetal,"" the boys' journey embroils them in a growing conspiracy that threatens the fate of the world."
1,45309,Mahou Shoujo Madoka★Magica,"Award Winning, Drama, Suspense","Madoka Kaname and Sayaka Miki are regular middle school girls with regular lives, but all that changes when they encounter Kyuubey, a cat-like magical familiar, and Homura Akemi, the new transfer student.\n\nKyuubey offers them a proposition: he will grant any one of their wishes and in exchange, they will each become a magical girl, gaining enough power to fulfill their dreams. However, Homura Akemi, a magical girl herself, urges them not to accept the offer, stating that everything is not what it seems.\n\nA story of hope, despair, and friendship, Mahou Shoujo Madoka★Magica deals with the difficulties of being a magical girl and the price one has to pay to make a dream come true."
2,42231,Steins;Gate,"Drama, Sci-Fi, Suspense","Eccentric scientist Rintarou Okabe has a never-ending thirst for scientific exploration. Together with his ditzy but well-meaning friend Mayuri Shiina and his roommate Itaru Hashida, Rintarou founds the Future Gadget Laboratory in the hopes of creating technological innovations that baffle the human psyche. Despite claims of grandeur, the only notable ""gadget"" the trio have created is a microwave that has the mystifying power to turn bananas into green goo.\n\nHowever, when Rintarou decides to attend neuroscientist Kurisu Makise's conference on time travel, he experiences a series of strange events that lead him to believe that there is more to the ""Phone Microwave"" gadget than meets the eye. Apparently able to send text messages into the past using the microwave, Rintarou dabbles further with the ""time machine,"" attracting the ire and attention of the mysterious organization SERN.\n\nDue to the novel discovery, Rintarou and his friends find themselves in an ever-present danger. As he works to mitigate the damage his invention has caused to the timeline, he is not only fighting a battle to save his loved ones, but also one against his degrading sanity."
3,17696,Chihayafuru,"Drama, Sports","As a child, Chihaya Ayase had only one dream: to see her elder sister Chitose become Japan's most successful model. However, upon defending her ostracised classmate Arata Wataya from his bully—Chihaya's childhood friend Taichi Mashima—she discovers the world of competitive karuta and soon becomes enamoured with the sport.\n\nBased on the Ogura Hundred Poets anthology, this card game where poems are studied requires excellent memory, agility, and a tremendous endurance from the players. Full of hope, Chihaya joins the Shiranami Society together with the newly reconciled Arata and Taichi, embarking on an exciting journey for the title awarded to the top-ranked female player—Queen of Karuta.\n\nSince middle school, 